In [0]:
import requests
import datetime
import json

In [0]:
class Collector:

    def __init__(self, url):
        self.url = url
        self.instance_name = url.strip("/").split("/")[-1]

    def get_endpoint(self, **kwargs):
        resp = requests.get(self.url, params=kwargs)
        return resp
    
    def save_data(self, data):
        now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S.%f")
        data["date_ref"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        filename = f"/Volumes/workspace/default/pokemon_volume/{self.instance_name}/json/{now}.json"
        with open(filename, "w") as open_file:
            json.dump(data, open_file)

    def get_n_save(self, **kwargs):
        resp = self.get_endpoint(**kwargs)
        if resp.status_code == 200:
            data = resp.json()
            self.save_data(data)
            return data
        else:
            return {}
        
    def auto_exec(self, limit=100):
        offset = 0
        while True:
            print("Já foram encontrados " + str(offset) + " pokémons. Buscando os próximos "+ str(limit) + "...")
            data = self.get_n_save(limit=limit, offset=offset)
            if data["next"] == None:
                break
            offset += limit

In [0]:
url = "https://pokeapi.co/api/v2/pokemon/"
collector = Collector(url)
collector.auto_exec()

In [0]:
df = spark.read.json("/Volumes/workspace/default/pokemon_volume/pokemon/json/")
display(df)